In [13]:
from keras import layers
import tensorflow as tf
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)


df = pd.read_csv('../csv_dateien/starcraftFinalcsvs/allRunsButOnlyTerran.csv')

conditions = [
    (df['fraction'] == "Protoss"), #1
    (df['fraction'] == "Terraner"), #2
    (df['fraction'] == "Zerg"), #3
    (df['fraction'] == "T"), #2
]

values = [1, 2, 3, 2]

df["fraction"] = np.select(conditions, values)
df=df[df["total_army"]>0]
df=df[df["fraction"]==2]
df=df[df["supply"]<=200]
df["ratio"] = df["total_workers"]/df["total_army"]

dfPrepSample = df.drop(columns=["player","winner","replay_filename", "Unnamed: 0", "total_workers", "total_army_value","fraction"])
len(dfPrepSample)


3009

In [14]:
# optimizer with scaling and lin reg

import warnings
import gurobipy as gp
import matplotlib.pyplot as plt
from sklearn import tree
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
from sklearn.compose import make_column_transformer
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr



# ,player,total_workers,total_army_value,total_army,fraction,winner,replay_filename
dfOpt = dfPrepSample.drop(columns=[
                               "ratio"])

dfOpt.head


<bound method NDFrame.head of       resource_mining  total_army  supply  time
1               28154        96.0   174.0   836
6               40998       120.0   196.0   934
15              13965        45.0   117.0   583
19               9595        26.0    62.0   449
24              38030        98.0   165.0  1102
...               ...         ...     ...   ...
5888             7364        17.0    69.0   381
5889            25845        80.0   173.0   878
5893            26351        70.0   157.0   795
5894            24263        76.0   165.0   754
5895            22655        40.0   125.0   726

[3009 rows x 4 columns]>

In [15]:


# create variables
# create Model

m = gp.Model("Balance optimizer")
gppd.set_interactive()

r = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="ratio")

a = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="total_army")
s = gppd.add_vars(m, pandas_obj=dfOpt,
                  name="supply")
w = m.addVar(name="workers", vtype=gp.GRB.INTEGER)

m.update()
dfOpt = dfOpt.drop(columns=["total_army"])

feats = dfOpt
feats = pd.concat([r, feats], axis=1)
feats.head


<bound method NDFrame.head of                          ratio  resource_mining  supply  time
1        <gurobi.Var ratio[1]>            28154   174.0   836
6        <gurobi.Var ratio[6]>            40998   196.0   934
15      <gurobi.Var ratio[15]>            13965   117.0   583
19      <gurobi.Var ratio[19]>             9595    62.0   449
24      <gurobi.Var ratio[24]>            38030   165.0  1102
...                        ...              ...     ...   ...
5888  <gurobi.Var ratio[5888]>             7364    69.0   381
5889  <gurobi.Var ratio[5889]>            25845   173.0   878
5893  <gurobi.Var ratio[5893]>            26351   157.0   795
5894  <gurobi.Var ratio[5894]>            24263   165.0   754
5895  <gurobi.Var ratio[5895]>            22655   125.0   726

[3009 rows x 4 columns]>

In [16]:
sc_model = tf.keras.models.load_model('sc_ts_model.h5')


In [17]:


m.setObjective(a.sum(), gp.GRB.MAXIMIZE) # Zielfunktion
"""
gppd.add_constrs(m, ((av - 50*w)/100), gp.GRB.GREATER_EQUAL, a) # Gleichung 2
"""
gppd.add_constrs(m, r * a, gp.GRB.EQUAL, w)
gppd.add_constrs(m, w + a, gp.GRB.GREATER_EQUAL, 0) # Gleichung 3
gppd.add_constrs(m, w + a, gp.GRB.LESS_EQUAL, s) # Gleichung 4
m.addConstr(w, gp.GRB.GREATER_EQUAL, 1)
m.update()

pred_constr = add_predictor_constr(m, sc_model, feats, a) # Einfügen des Regressiosmodells
pred_constr.print_stats()
m.Params.NonConvex = 2
m.optimize()


Model for sequential1:
45135 variables
21063 constraints
18054 general constraints
Input has shape (3009, 4)
Output has shape (3009, 1)

--------------------------------------------------------------------------------
Layer           Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
dense0             (3009, 3)        18054         9027            0         9027 (relu)

dense1             (3009, 3)        18054         9027            0         9027 (relu)

dense2             (3009, 1)            0         3009            0            0

--------------------------------------------------------------------------------
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize 

In [18]:


outputs = pred_constr.input_values
pred_constr.input_values



GurobiError: Unable to retrieve attribute 'X'

In [ ]:

solution = pd.DataFrame()

solution["ratio"] = r.gppd.X
solution["supply"] = s.gppd.X
solution["army"] = a.gppd.X
solution["worker"] = w
solution


In [ ]:

custom_palette = ["#dd8452","#4c72b0",  "#55a868"]
sns.set_theme(style="ticks")
# Show the joint distribution using kernel density estimation
g = sns.jointplot(
    data=solution, palette=custom_palette,
    x="ratio", y="supply",  # ylim=[None, 150],
    kind="kde", height=10
)